# FeedForward 🥸

#### Libraries

In [ ]:
import utilities as ff
from matplotlib import pyplot as plt
import numpy as np
import pathlib


In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import *
from keras.models import *
from keras.losses import *
from keras.optimizers import *
from keras.utils import *


#### Dataset

In [ ]:
batch_size = 42
img_width, img_height = 128, 128
color_mode = "grayscale"
no_classes = 2
seed = 42


In [ ]:
train, test = ff.ready_to_be_used_dataset(
    batch_size,
    image_squared_size=img_width,
    color_mode=color_mode,
)


In [ ]:
image_batch, labels_batch = next(iter(train))
print(image_batch.shape, labels_batch.shape)


In [ ]:
data_dir = pathlib.Path("downloads\CatsDogs")


In [ ]:
ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    color_mode=color_mode,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)


### Naive model

#### Model definition

In [ ]:
model = Sequential(
    [
        Flatten(input_shape=(img_width, img_height, 1)),
        Dense(100, activation="relu"),
        Dense(2, activation="softmax"),
    ]
)
model.summary()


In [ ]:
plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)


In [ ]:
model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


#### Hyperparameters tuning

In [ ]:
history = model.fit(train, validation_data=test, epochs=5)


In [ ]:
ff.performance_plot(history)


#### Risk estimation

In [ ]:
def_model= Sequential(
    [
        Flatten(input_shape=(img_width, img_height, 1)),
        Dense(100, activation="relu"),
        Dense(2, activation="softmax"),
    ]
)

In [ ]:
ff.five_fold_cross_validation(def_model,ds,no_epochs=5)

### Many neurons model

#### Model definition

In [ ]:
model = tf.keras.Sequential(
    [
        Flatten(input_shape=(img_width, img_height, 3)),
        Dense(2000, activation="relu"),
        Dense(2, activation="softmax"),
    ]
)
model.summary()


In [ ]:
plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)


In [ ]:
model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


#### Training

In [ ]:
history = model.fit(train, validation_data=test, epochs=5)


#### Performance evaluation

In [ ]:
ff.performance_plot(history)


### Many layers model

#### Model definition

In [ ]:
model = Sequential(
    [
        Flatten(input_shape=(img_width, img_height, 3)),
        Dense(1500, activation="relu"),
        Dense(500, activation="relu"),
        Dense(2, activation="softmax"),
    ]
)
model.summary()


In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)


In [ ]:
model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


#### Training

In [ ]:
history = model.fit(train, validation_data=test, epochs=5)


#### Performance evaluation

In [ ]:
ff.performance_plot(history)
